In [53]:
# CHANGE THESE
from pathlib import Path
CONFIG_DIR = Path("experiment_configs/flickr-reverse-image-search_caltech-256_02-24-2024.yaml")
HUGGINGFACE_MODEL = 'valhalla/distilbart-mnli-12-1'#"valhalla/distilbart-mnli-12-1" "facebook/bart-large-mnli"
NUM_THREADS = 4
NUM_INSTANCES = 1
INSTANCE_TYPE = "ml.p3.2xlarge"

In [18]:
import sys
import os
import zipfile
from pathlib import Path
import pandas as pd
import scipy.special as sp
import numpy as np
from PIL import Image
import torch
import json
from pathlib import Path
import argparse
import pickle
import traceback
from transformers import pipeline
import yaml 
from multiprocessing import Pool
from sagemaker.huggingface import HuggingFaceModel
import urllib, time
from botocore.exceptions import ClientError

script_path = Path(os.path.dirname(os.path.abspath(sys.argv[0])))
base_path = script_path.parent.absolute()
sys.path.append(base_path / 'cp')
sys.path.append(base_path / 'utils')
from utils.pets_classes import PETS_CLASSES, PETS_GENERIC_CLASSES
from utils.fitz17k_classes import FITZ17K_CLASSES, FITZ17K_GENERIC_CLASSES
from utils.medmnist_classes import MEDMNIST_CLASSES, MEDMNIST_GENERIC_CLASSES
from utils.imagenet_classes import IMAGENET_CLASSES, IMAGENET_GENERIC_CLASSES
from utils.caltech256_classes import CALTECH256_CLASSES, CALTECH256_GENERIC_CLASSES

config = {}
with open(CONFIG_DIR, "r") as yaml_file:
    config = yaml.safe_load(yaml_file)

for k, v in config.items():
    if (k[-4:] == '_dir'):
        config[k] = Path(v)

CONTEXT_DIRECTORY = Path("/home/sagemaker-user") / config['reverse_image_store_dir'].name
IMAGE_PLAUSIBILITIES = Path("/home/sagemaker-user") / config['image_plausibility_store_dir'].name
CALIB_IMAGE_DIRECTORY = Path("/home/sagemaker-user") / config['scraping_store_dir'].name
DATASET = config['dataset']

if DATASET == 'MedMNIST':
    LABELS = MEDMNIST_CLASSES
    PSEUDO_LABELS = MEDMNIST_GENERIC_CLASSES
elif DATASET == 'FitzPatrick17k':
    LABELS = FITZ17K_CLASSES
    PSEUDO_LABELS = FITZ17K_GENERIC_CLASSES
elif DATASET == 'OxfordPets':
    LABELS = PETS_CLASSES
    PSEUDO_LABELS = PETS_GENERIC_CLASSES
elif DATASET == 'ImageNet':
    LABELS = IMAGENET_CLASSES
    PSEUDO_LABELS = IMAGENET_GENERIC_CLASSES
elif DATASET == "Caltech256":
    LABELS = CALTECH256_CLASSES
    PSEUDO_LABELS = CALTECH256_GENERIC_CLASSES
else:
    LABELS = None

In [19]:
import sagemaker
import boto3
from sagemaker.async_inference.async_inference_config import AsyncInferenceConfig
from sagemaker.s3 import s3_path_join
from sagemaker.async_inference.waiter_config import WaiterConfig

sess = sagemaker.Session()
sagemaker_session_bucket = sess.default_bucket()
try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

print(f"sagemaker role arn: {role}")
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker role arn: arn:aws:iam::475664224131:role/service-role/AmazonSageMaker-ExecutionRole-20240304T212097
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker role arn: arn:aws:iam::475664224131:role/service-role/AmazonSageMaker-ExecutionRole-20240304T212097
sagemaker bucket: sagemaker-us-east-2-475664224131
sagemaker session region: us-east-2


In [20]:
os.system(f"aws s3 cp s3://sagemaker-datasets-hwei0/{config['reverse_image_store_dir'].name}.zip /home/sagemaker-user")
os.system(f"aws s3 cp s3://sagemaker-datasets-hwei0/{config['scraping_store_dir'].name}.zip /home/sagemaker-user")

download: s3://sagemaker-datasets-hwei0/flickr_web_scraping_0224_selenium_reverse-image-search-selenium_NEW-CAPTION-METHOD_caltech-256_25size_caption-results.zip to ../flickr_web_scraping_0224_selenium_reverse-image-search-selenium_NEW-CAPTION-METHOD_caltech-256_25size_caption-results.zip
download: s3://sagemaker-datasets-hwei0/flickr_web_scraping_0224_selenium_reverse-image-search-selenium_NEW-CAPTION-METHOD_caltech-256_25size.zip to ../flickr_web_scraping_0224_selenium_reverse-image-search-selenium_NEW-CAPTION-METHOD_caltech-256_25size.zip


0

In [21]:
with zipfile.ZipFile(Path("/home/sagemaker-user") / f"{config['reverse_image_store_dir'].name}.zip", 'r') as zip_ref:
    zip_ref.extractall(Path("/home/sagemaker-user"))
with zipfile.ZipFile(Path("/home/sagemaker-user") / f"{config['scraping_store_dir'].name}.zip", 'r') as zip_ref:
    zip_ref.extractall(Path("/home/sagemaker-user"))

In [28]:
# Hub model configuration <https://huggingface.co/models>
hub = {
  'HF_MODEL_ID':HUGGINGFACE_MODEL, # model_id from hf.co/models
  'HF_TASK':'zero-shot-classification'                           # NLP task you want to use for predictions
}
# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   env=hub,  # path to your trained sagemaker model
   role=role, # iam role with permissions to create an Endpoint
   transformers_version="4.26", # transformers version used
   pytorch_version="1.13", # pytorch version used
   py_version="py39", # python version of the DLC
)
# Create async config
async_config = AsyncInferenceConfig(
    output_path=s3_path_join("s3://",sagemaker_session_bucket,"async_inference/output")
)
# Create waiter config
config = WaiterConfig(
  max_attempts=5, 
  delay=10 
  )
# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
   initial_instance_count=NUM_INSTANCES,
   instance_type=INSTANCE_TYPE,
   async_inference_config=async_config
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
-----------!

In [54]:
def get_output(output_location):
    output_url = urllib.parse.urlparse(output_location)
    bucket = output_url.netloc
    key = output_url.path[1:]
    while True:
        try:
            return sess.read_s3_file(bucket=output_url.netloc, key_prefix=output_url.path[1:])
        except ClientError as e:
            if e.response["Error"]["Code"] == "NoSuchKey":
                time.sleep(5)
                continue
            raise
def scores_converter(scores, labels):
    dict_scores = {}
    n = len(scores['labels'])
    for i in range(0, n):
        label = scores['labels'][i]
        dict_scores[label] = scores['scores'][i]
    score_vals = []
    for label in labels:
        score_vals.append(dict_scores[label])
    return score_vals

def get_score_async(predict_dict):
    request = predictor.predict_async(predict_dict)
    res = json.loads(get_output(request.output_path))
    scores = scores_converter({"scores": res[0]['scores'], "labels": res[0]['labels']}, list(LABELS.values()))
    return scores

def score_generation(label):
    print("Beginning Score Generation: {label}".format(label=label))
    os.makedirs(IMAGE_PLAUSIBILITIES / label, exist_ok=True)
    n = 0
    for file in os.listdir(CONTEXT_DIRECTORY / label):
        # Load captions 
        if file.endswith("_debug.pkl") or file.endswith("events.log"): continue
        try:
            with open(CALIB_IMAGE_DIRECTORY / label / (file.split('.')[0]+'.caption'), 'r') as read:
                title = "\n".join([line.rstrip() for line in read])
        except:
            print('ERROR PKL LOAD')
            print(traceback.format_exc())
            continue
        captions = pickle.load(open(CONTEXT_DIRECTORY / label / file, 'rb'))
        if len(captions) <= 1: 
            print('ERROR # CAPTIONS:' + str(captions))
            continue
        # Main Score 
        main_score = get_score_async({
            "inputs": [title],
            "parameters": {'candidate_labels': list(LABELS.values()), 'multi_label': True, 'use_cache':True}
        })
        main_score = torch.tensor(main_score)
        # Second Score
        second_score = []
        second_search = captions[0:min(10, len(captions))]
        label_set = list(set(PSEUDO_LABELS.values()))
        for caption in second_search:
            score = get_score_async({
                "inputs":[caption], 
                "parameters": {'candidate_labels': label_set, 'multi_label':True, 'use_cache':True}
            })
            second_score.append(score)
            
        second_score = [torch.tensor(score) for score in second_score]
        second_score = torch.stack(second_score)
        torch.save(main_score, IMAGE_PLAUSIBILITIES / label / (file.split(".")[0] + '_main'))
        torch.save(second_score, IMAGE_PLAUSIBILITIES / label / (file.split(".")[0] + '_second'))
        print('a')
        n += 1
        if n >= 10: break

    sys.stdout.flush()


In [50]:
# res = predictor.predict_async({
#             "inputs": ["an image of a chair"],
#             "parameters": {'candidate_labels': list(LABELS.values()), 'multi_label': True, 'use_cache':True}
#         })

In [51]:
# res_out = json.loads(get_output(res.output_path))
# scores_converter({'scores': res_out[0]['scores'], 'labels': res_out[0]['labels']}, list(LABELS.values()))

In [52]:
# res_out

In [55]:


# Encode Labels
#label_embed = model.encode([label for label in LABELS.values()])
labels = [label.split(',')[0] for label in LABELS.values()]
print(labels)
#pseudo_embed = model.encode([label for label in PSEUDO_LABELS.values()])
# Loop through caption folders
dir_labels = []
for label in os.listdir(CONTEXT_DIRECTORY):
    try: 
        int(label)
    except:
        continue
    if label.endswith("events.log"): 
        continue
    dir_labels.append(label)

with Pool(processes=NUM_THREADS) as executor:
    executor.map(score_generation, dir_labels)

['ak47', 'american flag', 'backpack', 'baseball bat', 'baseball glove', 'basketball hoop', 'bat animal', 'bathtub', 'bear', 'beer mug', 'billiards', 'binoculars', 'birdbath', 'blimp', 'bonsai', 'boom box', 'bowling ball', 'bowling pin', 'boxing glove', 'brain', 'breadmaker', 'buddha', 'bulldozer', 'butterfly', 'cactus', 'cake', 'calculator', 'camel', 'cannon', 'canoe', 'car tire', 'cartman', 'cd', 'centipede', 'cereal box', 'chandelier', 'chess board', 'chimp', 'chopsticks', 'cockroach', 'coffee mug', 'coffin', 'coin', 'comet', 'computer keyboard', 'computer monitor', 'computer mouse', 'conch', 'cormorant', 'covered wagon', 'cowboy hat', 'crab', 'desk globe', 'diamond ring', 'dice', 'dog', 'dolphin', 'doorknob', 'drinking straw', 'duck', 'dumb bell', 'eiffel tower', 'electric guitar', 'elephant', 'elk', 'ewer', 'eyeglasses', 'fern', 'fighter jet', 'fire extinguisher', 'fire hydrant', 'fire truck', 'fireworks', 'flashlight', 'floppy disk', 'football helmet', 'french horn', 'fried egg', 


KeyboardInterrupt



In [56]:
# delete endpoint
predictor.delete_model()
predictor.delete_endpoint()

In [ ]:
os.system(f"aws s3 cp {IMAGE_PLAUSIBILITIES.resolve()} s3://sagemaker-datasets-hwei0 --recursive")